Imports and constants

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
from pathlib import Path
from tqdm import tqdm

import torch
import torch.utils.data as thd
import torch.nn as nn

from sklearn.metrics import fbeta_score, precision_score, recall_score

from scipy.ndimage.filters import gaussian_filter1d

import matplotlib.pyplot as plt

from collections import defaultdict
import os

import gc

BATCH_SIZE = 256
BUFFER = 10  # Buffer size in both dimensions: x and y. Effective patch size is [BUFFER * 2 + 1, BUFFER * 2 + 1, Z_DIM].
SLICES = 65
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 0.005
Z_START = 25
Z_END = 45
Z_STEP = 2
Z_DIM = (Z_END - Z_START) // Z_STEP

MAX_TRAIN_STEPS = 55000
MAX_VAL_STEPS = 1000
PRINT_EVERY = 20000000

First, let's see if we can fit all one full fragment into memory at once.

In [ ]:
def pad_array(array):
    padding = (
        (BUFFER, BUFFER),
        (BUFFER, BUFFER),
    )
    return np.pad(array, padding)

def load_fragment_surface(fragment, split='train'):
    print("Loading fragment %s surface" % fragment)
    surface_path = Path("/kaggle/input/vesuvius-challenge-ink-detection/%s/%s/surface_volume" % (split, fragment))
    return np.array([
        (pad_array(np.array(Image.open(f))) / (2 ** 16)).astype('float16')
        for f in tqdm(sorted(surface_path.rglob("*.tif"))[Z_START:Z_END:Z_STEP])
    ])

def load_mask(fragment, split='train'):
    print("Loading fragment %s mask" % fragment)
    mask_path = Path("/kaggle/input/vesuvius-challenge-ink-detection/%s/%s/mask.png" % (split, fragment))
    return pad_array(np.array(Image.open(mask_path)))

def load_inklabels(fragment, split='train'):
    print("Loading fragment %s labels" % fragment)
    inklabels_path = Path("/kaggle/input/vesuvius-challenge-ink-detection/%s/%s/inklabels.png" % (split, fragment))
    return pad_array(np.array(Image.open(inklabels_path)))

In [ ]:
class SingleFragmentDataset(thd.Dataset):
    def __init__(self, fragment, is_test=False):
        self.is_test = is_test
        split = 'test' if is_test else 'train'
        self.surface = load_fragment_surface(fragment, split)
        print(self.surface.dtype)
        self.mask = load_mask(fragment, split)
        self.inklabels = load_inklabels(fragment, split) if not is_test else None
        self.pixels = np.stack(np.where(self.mask == 1), axis=1)
    
    def __len__(self):
        return self.pixels.shape[0]
    
    def get_pixel_number(self, y, x):
        return 1 + y * self.surface.shape[1] + x
    
    def __getitem__(self, index):
        y, x = self.pixels[index]
        y_start = y - BUFFER
        y_end = y + BUFFER + 1
        x_start = x - BUFFER
        x_end = x + BUFFER + 1
        patch_surface = np.s_[:, y_start:y_end, x_start:x_end]
        patch_labels = np.s_[y_start:y_end, x_start:x_end]
        surface = self.surface[patch_surface].astype(np.float32)
        labels = self.inklabels[y, x].reshape((1, )).astype(np.float32) if not self.is_test else None
        return (surface, labels) if not self.is_test else (surface, self.get_pixel_number(y, x))

In [ ]:
%%time
train_dataset = SingleFragmentDataset(1)
print(train_dataset.surface.shape)

In [ ]:
# plt.hist(train_dataset.surface.flatten())

In [ ]:
from operator import itemgetter

from pympler import tracker

print(gc.collect())

mem = tracker.SummaryTracker()
print(sorted(mem.create_summary(), reverse=True, key=itemgetter(2))[:10])

In [ ]:
train_loader = thd.DataLoader(train_dataset, BATCH_SIZE, shuffle=True)

Define the model

In [ ]:
convnet = nn.Sequential(
    nn.Conv2d(Z_DIM, 32, kernel_size=3, stride=1, dilation=1, padding='same'),
    nn.Dropout(p=0.2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Conv2d(32, 16, kernel_size=3, stride=1, dilation=1, padding='same'),
    nn.Dropout(p=0.2),
    nn.ReLU(),
    nn.BatchNorm2d(16),
    nn.Conv2d(16, 8, kernel_size=3, stride=1, dilation=1, padding='same'),
    nn.Dropout(p=0.2),
    nn.ReLU(),
    nn.BatchNorm2d(8),
    nn.Conv2d(8, 1, kernel_size=3, stride=1, dilation=1, padding='same'),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear((2 * BUFFER + 1) ** 2, 128),
    nn.Dropout(p=0.2),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.Dropout(p=0.2),
    nn.ReLU(),
    nn.Linear(64, 1),
    nn.Sigmoid()
).to(DEVICE)

Train the model

In [ ]:
%%time
losses = []
accs = []
fbetas = []

convnet.train()
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(convnet.parameters(), lr=LEARNING_RATE)
for i, (xs, ys) in enumerate(pbar := tqdm(train_loader)):
    if i > MAX_TRAIN_STEPS:
        break
    optimizer.zero_grad()
    outputs = convnet(xs.to(DEVICE))
    if i % PRINT_EVERY == PRINT_EVERY - 1:
        print(outputs)
        plt.plot(gaussian_filter1d(losses, sigma=10), label='loss')
        plt.plot(gaussian_filter1d(accs, sigma=10), label='accs')
        plt.plot(gaussian_filter1d(fbetas, sigma=10), label='fbetas')
        plt.legend()
    loss = criterion(outputs, ys.to(DEVICE))
    pred_ink = outputs.detach().gt(0.4).cpu().int()
    accuracy = (pred_ink == ys).sum().float().div(ys.size(0))
    fbeta = fbeta_score(ys.view(-1).numpy(), pred_ink.view(-1).numpy(), beta=0.5)
    pbar.set_postfix({"loss": loss, "acc": accuracy, "fbeta": fbeta})
    loss.backward()
    optimizer.step()
    
    fbetas.append(fbeta)
    losses.append(loss.detach().cpu().float())
    accs.append(accuracy)

In [ ]:
plt.plot(gaussian_filter1d(losses, sigma=10), label='loss')
plt.plot(gaussian_filter1d(accs, sigma=10), label='accs')
plt.plot(gaussian_filter1d(fbetas, sigma=10), label='fbetas')
plt.legend()

Validate the model

In [ ]:
del train_loader
del train_dataset
gc.collect()

convnet.eval()
val_dataset = SingleFragmentDataset(3)
val_loader = thd.DataLoader(val_dataset, BATCH_SIZE, shuffle=True)

def evaluate(convnet, val_loader, threshold):
    print("Evaluating for threshold %f" % threshold)
    losses = []
    accs = []
    fbetas = []
    precisions = []
    recalls = []

    for i, (xs, ys) in enumerate(pbar := tqdm(val_loader)):
        if i > MAX_VAL_STEPS:
            break
        outputs = convnet(xs.to(DEVICE))
        loss = criterion(outputs, ys.to(DEVICE))
        pred_ink = outputs.detach().gt(threshold).cpu().int()
        accuracy = (pred_ink == ys).sum().float().div(ys.size(0))
        fbeta = fbeta_score(ys.view(-1).numpy(), pred_ink.view(-1).numpy(), beta=0.5)
        precision = precision_score(ys.view(-1).numpy(), pred_ink.view(-1).numpy())
        recall = recall_score(ys.view(-1).numpy(), pred_ink.view(-1).numpy())
        pbar.set_postfix({"loss": loss, "acc": accuracy, "fbeta": fbeta})

        fbetas.append(fbeta)
        losses.append(loss.detach().cpu().float())
        accs.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
    
    return np.mean(losses), np.mean(accs), np.mean(fbetas), np.mean(precisions), np.mean(recalls)


def evaluate_for_thresholds():
    thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    best_fbeta = 0.
    best_threshold = 0
    
    fbetas = []
    precisions = []
    recalls = []

    for threshold in thresholds:
        _, _, fbeta, precision, recall = evaluate(convnet, val_loader, threshold)
        if fbeta > best_fbeta:
            best_fbeta = fbeta
            best_threshold = threshold
        fbetas.append(fbeta)
        precisions.append(precision)
        recalls.append(recall)
    
    plt.plot(thresholds, fbetas, label="fbeta")
    plt.plot(thresholds, precisions, label="precision")
    plt.plot(thresholds, recalls, label="recall")
    plt.legend()
    
    return best_threshold

best_threshold = evaluate_for_thresholds()
print("Best threshold:", best_threshold)

Generate test predictions

In [ ]:
del val_loader
del val_dataset
gc.collect()

submission = defaultdict(list)

for fragment in ['a', 'b']:
    pixels_with_ink = []
    print(f"Generating predictions for fragment {fragment}")
    test_dataset = SingleFragmentDataset(fragment, is_test=True)
    test_loader = thd.DataLoader(test_dataset, BATCH_SIZE, shuffle=True)
    for (xs, ys) in (pbar := tqdm(test_loader)):
        output = convnet(xs.to(DEVICE))
        pred_ink = outputs.detach().gt(best_threshold).flatten().cpu().int()
        pred_ink = pred_ink[:len(ys)]
        pixels_with_ink += ys[pred_ink == 1].int().tolist()
    pixels_with_ink.sort()
    
    submission["Id"].append(fragment)
    submission["Predicted"].append(' '.join(list(map(lambda p: "%s 1" % p, pixels_with_ink))))
    
pd.DataFrame.from_dict(submission).to_csv("/kaggle/working/submission.csv", index=False)

In [ ]:
pd.DataFrame.from_dict(submission)